In [1]:
from elasticsearch import Elasticsearch

es = Elasticsearch("http://localhost:9200")

In [6]:
es.indices.delete(index="vector_index", ignore=404)

/var/folders/mq/fxmjb3ss6mg8hjxwd1sy_zbc0000gn/T/ipykernel_5712/1275999398.py:1: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.delete(index="vector_index", ignore=404)


ObjectApiResponse({'error': {'root_cause': [{'type': 'index_not_found_exception', 'reason': 'no such index [vector_index]', 'resource.type': 'index_or_alias', 'resource.id': 'vector_index', 'index_uuid': '_na_', 'index': 'vector_index'}], 'type': 'index_not_found_exception', 'reason': 'no such index [vector_index]', 'resource.type': 'index_or_alias', 'resource.id': 'vector_index', 'index_uuid': '_na_', 'index': 'vector_index'}, 'status': 404})

In [9]:
# 1. 定义索引结构

body = {
    "mappings": {
        "properties": {
            "name": {
                "type": "text"
            },
            "embedding": {
                "type": "dense_vector",
                "dims": 3
            }
        }
    }
}

In [10]:
# 2. 创建索引

es.indices.create(index="vector_index", body=body)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'vector_index'})

In [11]:
# 3. 批量插入数据
from elasticsearch import helpers

actions = [
    {"_index": "vector_index",
     "_source": {"name": "苹果",
                 "embedding": [0.2, 0.1, 0.4]}},
    {"_index":"vector_index",
     "_source":{"name":"小船",
                "embedding":[0.7,0.2,0.6]}},
    {"_index":"vector_index",
     "_source":{"name":"香蕉",
                "embedding":[0.3,0.1,0.3]}}
]

helpers.bulk(es, actions=actions)

(3, [])

In [12]:
# 4. 查询所有记录
result = es.search(index="vector_index", body={"query": {"match_all": {}}})

print(result)

{'took': 5, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.0, 'hits': [{'_index': 'vector_index', '_id': 'o7SiFJMBKr7psItEE-Up', '_score': 1.0, '_source': {'name': '苹果', 'embedding': [0.2, 0.1, 0.4]}}, {'_index': 'vector_index', '_id': 'pLSiFJMBKr7psItEE-Ur', '_score': 1.0, '_source': {'name': '小船', 'embedding': [0.7, 0.2, 0.6]}}, {'_index': 'vector_index', '_id': 'pbSiFJMBKr7psItEE-Ur', '_score': 1.0, '_source': {'name': '香蕉', 'embedding': [0.3, 0.1, 0.3]}}]}}


In [15]:
# 5. 根据向量计算相似性

temp = "橘子"
temp_emb = [0.2, 0.2, 0.4]

query = {
    "query": {
        "script_score": {
            "query": {
                "match_all": {}
            },
            "script": {
                "source": "cosineSimilarity(params.queryVector, 'embedding') + 1.0",
                "params": {
                    "queryVector": temp_emb
                }
            }
        }
    }
}

response = es.search(index="vector_index", body=query)

print(response)

{'took': 2, 'timed_out': False, '_shards': {'total': 1, 'successful': 1, 'skipped': 0, 'failed': 0}, 'hits': {'total': {'value': 3, 'relation': 'eq'}, 'max_score': 1.9799578, 'hits': [{'_index': 'vector_index', '_id': 'o7SiFJMBKr7psItEE-Up', '_score': 1.9799578, '_source': {'name': '苹果', 'embedding': [0.2, 0.1, 0.4]}}, {'_index': 'vector_index', '_id': 'pbSiFJMBKr7psItEE-Ur', '_score': 1.9365858, '_source': {'name': '香蕉', 'embedding': [0.3, 0.1, 0.3]}}, {'_index': 'vector_index', '_id': 'pLSiFJMBKr7psItEE-Ur', '_score': 1.9087589, '_source': {'name': '小船', 'embedding': [0.7, 0.2, 0.6]}}]}}


In [17]:
response['hits']

{'total': {'value': 3, 'relation': 'eq'},
 'max_score': 1.9799578,
 'hits': [{'_index': 'vector_index',
   '_id': 'o7SiFJMBKr7psItEE-Up',
   '_score': 1.9799578,
   '_source': {'name': '苹果', 'embedding': [0.2, 0.1, 0.4]}},
  {'_index': 'vector_index',
   '_id': 'pbSiFJMBKr7psItEE-Ur',
   '_score': 1.9365858,
   '_source': {'name': '香蕉', 'embedding': [0.3, 0.1, 0.3]}},
  {'_index': 'vector_index',
   '_id': 'pLSiFJMBKr7psItEE-Ur',
   '_score': 1.9087589,
   '_source': {'name': '小船', 'embedding': [0.7, 0.2, 0.6]}}]}

In [18]:
response['hits']['hits']

[{'_index': 'vector_index',
  '_id': 'o7SiFJMBKr7psItEE-Up',
  '_score': 1.9799578,
  '_source': {'name': '苹果', 'embedding': [0.2, 0.1, 0.4]}},
 {'_index': 'vector_index',
  '_id': 'pbSiFJMBKr7psItEE-Ur',
  '_score': 1.9365858,
  '_source': {'name': '香蕉', 'embedding': [0.3, 0.1, 0.3]}},
 {'_index': 'vector_index',
  '_id': 'pLSiFJMBKr7psItEE-Ur',
  '_score': 1.9087589,
  '_source': {'name': '小船', 'embedding': [0.7, 0.2, 0.6]}}]

PUT /hotel_info
{
    "settings": {
        "number_of_shards": 1,
        "number_of_replicas": 1
    },
    "mappings": {
        "properties": {
            "hotel_id": {
                "type": "integer"
            },
            "name": {
                "type": "text",
                "analyzer": "ik_smart"
            },
            "city": {
                "type": "keyword"
            },
            "star": {
                "type": "integer"
            },
            "open_year": {
                "type": "date"
            },
            "room_quantity": {
                "type": "integer"
            },
            "zone": {
                "type": "text",
                "analyzer": "ik_max_word"
            },
            "area": {
                "type": "keyword"
            },
            "address": {
                "type": "text",
                "analyzer": "ik_max_word"
            },
            "location": {
                "type": "geo_point"
            },
            "comment_total": {
                "type": "integer"
            },
            "total_score": {
                "type": "float"
            },
            "health_score": {
                "type": "float"
            },
            "environment_score": {
                "type": "float"
            },
            "service_score": {
                "type": "float"
            },
            "facilities_score": {
                "type": "float"
            },
            "bullet_screen_info": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "null_value": "NULL"
                    }
                }
            },
            "hotel_facilities_popular": {
                "type": "text"
            },
            "hotel_policy": {
                "type": "text"
            },
            "description": {
                "type": "text"
            },
            "traffic": {
                "type": "text"
            },
            "min_price": {
                "type": "float",
                "null_value": 0.0
            },
            "min_price_checkin_date": {
                "type": "date",
                "null_value": "1997-01-01"
            },
            "base_rooms": {
                "type": "text",
                "fields": {
                    "keyword": {
                        "type": "keyword",
                        "null_value": "NULL"
                    }
                }
            }
        }
    }
}

GET /hotel_info/_count

GET /hotel_info/search?_source=name,city,area
{
    "query": {
        "match_all: {}
    }
}

GET /hotel_info/search?source=name
{
    "query": {
        "match_all": {}
    },
    "from": 4,
    "size": 5
}

GET /hotel_info/search?source=name,city
{
    "profile": true,
    "from": 0,
    "size": 5,
    "query": {
        "match": {
            "city": "上海“
        }
    }
}

GET /hotel_info/_explain/0
{
    "query": {
        "match": {
            "city": "上海"
        }
    }
}

#### 布尔查询

- must查询，相当于逻辑查询中"与”查询
GET /hotel_info/search？source=name, area, min_price
{
    "query": {
        "bool": {
            "must": [
                {
                    "terms": {
                        "area.keyword": [
                            "黄浦区",
                            "静安区"
                        ]
                    }
                }，
                {
                    "range": {
                        "min_price": {
                            "gt": 300,
                            "It": 400
                        }
                    }
                }
            ]
        }
    }
}

- should查询，表示当前查询为“或”查询

- must not查询，表示当前查询为“非”查询

- filter查询即过滤查询，关注查询条件和文档是否匹配，不进行相关的打分计算

#### 全文搜索
全文搜索首先对查询词进行分析，然后根据查询词的分词结果构建查询，这里所说的全文指的是文本类型数据（text类型）
- match查询
说明：match查询是全文搜索的主要代表，对于最基本的math搜索来说，只要分词中的一个或者多个在文档中存在即可。

说明：match搜索可以设置operator参数，该参数决定文档按照分词后的词集合进行“与”还是“或”匹配。在默认情况下，该参数的值为“或”关系，即operator的值为or，这也解释了搜索结果中包含部分匹配的文档。如果希望各个词之间的匹配结果是“与”关系，则可以设置operator参数的值为and

- mutli-match查询
有时用户需要在多个字段中查询关键词，除了使用布尔查询封装多个match查询之外，可替代的方案是使用multi_match。可以在multi_match的query子句中组织数据匹配规则，并在fields子句中指定需要搜索的字段列表。
方式1：使用bool方式实现
GET /hotel_info/_search?_source=name,hotel_facilities_popular,hotel_facilities_full
{
  "query":{
    "bool":{
      "should":[
        {
          "match":{
            "hotel_facilities_popular":"24小时前台"   // 条件1
          }
        },
        {
          "match":{
            "hotel_facilities_full":"免费停车场" // 条件2
          }
        }
      ]
    }
  }
}

方式2：使用multi-match
GET /hotel_info/_search?_source=name,hotel_facilities_popular,hotel_facilities_full
{
  "query":{
    "multi_match":{
      "query":"免费停车场,24小时前台",
      "type":"best_fields", // 多个字段中，返回评分最高的
      "fields": ["hotel_facilities_popular","hotel_facilities_full"],
      "operator": "or"
    }
  }
}

- match_phrase查询
// match_phrase用于匹配短语，与match查询不同的是，match_phrase用于搜索确切的短语或邻近的词语。
// 可以设置match_phrase查询的slop参数，它用来调节匹配词之间的距离阈值。
// 示例2：设置match_phrase查询的slop参数，它用来调节匹配词之间的距离阈值
GET /hotel_info/_search?_source=name
{
  "query":{
    "match_phrase": {
      "name": {
        "query":"万豪酒店",
        "slop":4  // 调节匹配词之间的距离阈值
      }
    }
  }
}

按字段值排序
// ES提供了sort子句可以对数据进行排序。
// 使用sort子句一般是按照字段信息进行排序，不受相关性影响，而且打分步骤需要耗费一定的硬件资源和时间，因此默认情况下，不对文档进行打分。
// 使用sort排序分为两种类别，一种是按照字段值的大小进行排序，另一种是按照给定地理坐标的距离远近进行排序。
// 排序字段不可以是text等特殊类型，一般是整数类型和keyword类型。

// 5.3.1 按照普通字段值排序

// 示例1：搜索名称包含"万豪"的酒店，并对酒店按照价格进行升序排列
GET /hotel_info/_search?_source=name,min_price
{
  "query":{
    "match":{
      "name":"万豪"
    }
  },
  "sort":[
    {"min_price": {
      "order": "asc"
    }}
  ]
}

// 说明：文档的_score值和max_score都为null，这说明在默认情况下ES查询时使用sort对结果排序是不计算分数的。



// 示例2：搜索名称包含"万豪"或"全季"的酒店，并对酒店按照价格进行升序排列,再按照星级进行降序排列
GET /hotel_info/_search?_source=name,min_price,star
{
  "query":{
    "bool":{
      "should":[
        {
          "match":{ // 匹配1
            "name":"万豪"
          }
        },
        {
          "match":{ // 匹配2
            "name":"全季"
          }
        }
      ]
    }
  },
  "sort":[
    { // 条件1：按照价格升序排列,
      "min_price": {"order":"asc"}
    },
    { // 条件2：按照星级降序排列
      "star": {"order": "desc"}
    }
  ]
}


// 5.3.2 按照地理距离排序

// 使用geo_distance查询，配合sort可以指定另一种排序规则，即按照文档坐标与指定坐标的距离对结果进行排序。
// 除了可以指定升序或者降序排列外，还可以指定排序结果中sort子句中的距离的计量单位，默认值为km即千米。
// 在进行距离计算时，系统默认使用的算法为arc，该算法的特点是计算精准但是耗费时间较长，用户可以使用distance_type参数选择另一种计算速度快但经度略差的算法，名称为plane。

// 示例1：使用geo_distance查询5km范围内的酒店，并按照距离由近及远进行排序

GET /hotel_info/_search?_source=name,area,address
{
  "query":{
    "geo_distance":{
      "distance":"5km",  // 设置地理范围5公里
      "location":{ // 中心点坐标
        "lat":"31.276613",  // 维度
        "lon":"121.442561"  // 经度
      }
    }
  },
  "sort":[
    {
      "_geo_distance": {
        "location": { // 中心点坐标
          "lat": "31.276613",
          "lon": "121.442561"
        },
        "order": "asc",  // 按距离由近及远进行排序
        "unit": "km",  // 距离单位
        "distance_type": "plane"  // 计算算法，默认算法arc
      }
    }
  ]
}